# **LAB-2: Scalable Machine Learning and Deep Learning**

## **Paolo Teta & Ralfs Zangis**
---
Implement SBERT model.
The task which will be carried out are:

1.   Load and prepare the data
2.   Perform regression
3.   Perform classification
4.   Fine-tune BERT pre-trained model
5.   Evaluate the performance on a data set (Semantic
     Textual Similarity (STS) benchmark) 

Load the data and pre-process them through the bert-as-service tool, which allow to tokenize the datas used.



In [ ]:
!pip install tokenizers
!pip install transformers
!pip install wget

import csv
import wget
import os
import re
import json
import string
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
from tensorflow.keras.layers import Input
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig

max_len = 200

     |████████████████████████████████| 3.3 MB 4.0 MB/s 
     |████████████████████████████████| 3.1 MB 4.3 MB/s 
     |████████████████████████████████| 596 kB 46.6 MB/s 
     |████████████████████████████████| 61 kB 495 kB/s 
     |████████████████████████████████| 895 kB 40.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=6011af0142de532eec6b9f8b60386ce866e53e68efa80b1b55d685dbaddf8aa1
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [ ]:
!pip install sentence_transformers
from sentence_transformers.readers import InputExample

     |████████████████████████████████| 78 kB 3.0 MB/s 
     |████████████████████████████████| 1.2 MB 24.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=9d16c9353f36a010d160b15c2e5d4345c48762160956ae0bd349fd2eb2f4d10b
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers


In [ ]:
train_path = '/content/sts-train.csv'
train_samples=[]
with open(train_path, newline='') as train:
  columns=['title', 'type', 'year', 'id', 'value', 'sentenceA', 'sentenceB']
  df = csv.DictReader(train, delimiter='\t', fieldnames=columns, quoting=csv.QUOTE_NONE)
  for row in df:
     score = float(row['value']) / 2.5 - 1  # Normalize score to range -1 ... 1
     inp_example = InputExample(texts=[row['sentenceA'], row['sentenceB']], label=score)
     train_samples.append(inp_example)
     #print(inp_example)

In [ ]:
df_dev = pd.read_csv('/content/sts-dev.csv', sep='\t', header=None, error_bad_lines=False)
dev_samples=[]
with open('/content/sts-dev.csv', newline='') as train:
  columns=['title', 'type', 'year', 'id', 'value', 'sentenceA', 'sentenceB']
  df = csv.DictReader(train, delimiter='\t', fieldnames=columns, quoting=csv.QUOTE_NONE)
  for row in df:
     score = float(row['value']) / 2.5 - 1  # Normalize score to range -1 ... 1
     inp_example = InputExample(texts=[row['sentenceA'], row['sentenceB']], label=score)
     dev_samples.append(inp_example)
     #print(inp_example)

b'Skipping line 1041: expected 7 fields, saw 8\nSkipping line 1065: expected 7 fields, saw 8\nSkipping line 1082: expected 7 fields, saw 8\nSkipping line 1136: expected 7 fields, saw 8\nSkipping line 1149: expected 7 fields, saw 8\nSkipping line 1449: expected 7 fields, saw 9\nSkipping line 1450: expected 7 fields, saw 9\nSkipping line 1451: expected 7 fields, saw 9\nSkipping line 1452: expected 7 fields, saw 9\nSkipping line 1453: expected 7 fields, saw 9\nSkipping line 1454: expected 7 fields, saw 9\nSkipping line 1455: expected 7 fields, saw 9\nSkipping line 1456: expected 7 fields, saw 9\nSkipping line 1457: expected 7 fields, saw 9\nSkipping line 1458: expected 7 fields, saw 9\nSkipping line 1459: expected 7 fields, saw 9\nSkipping line 1460: expected 7 fields, saw 9\nSkipping line 1461: expected 7 fields, saw 9\nSkipping line 1462: expected 7 fields, saw 9\nSkipping line 1463: expected 7 fields, saw 9\nSkipping line 1464: expected 7 fields, saw 9\nSkipping line 1465: expected 7 f

In [ ]:
test_line = '/content/sts-test.csv'
#os.path.isfile(test_path)
test_samples=[]
with open('/content/sts-test.csv', newline='') as train:
  columns=['title', 'type', 'year', 'id', 'value', 'sentenceA', 'sentenceB']
  df = csv.DictReader(train, delimiter='\t', fieldnames=columns, quoting=csv.QUOTE_NONE)
  for row in df:
     score = float(row['value']) / 2.5 - 1  # Normalize score to range -1 ... 1
     inp_example = InputExample(texts=[row['sentenceA'], row['sentenceB']], label=score)
     test_samples.append(inp_example)
     #print(inp_example)

In [ ]:
!pip install sentence_transformers

from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

In [ ]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = TFBertModel.from_pretrained("bert-base-uncased")

In [ ]:
model_name = 'bert-base-uncased'
word_embedding_model = models.Transformer(model_name)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
train_batch_size = 16
learn_rate = 2e-5
num_epochs = 1

from datetime import datetime
model_save_path = './training_sts_'+model_name.replace("/", "-")+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [ ]:
# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

################################################################################
# creating custom model, by using mean pooling of the word embeddings given as input
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
################################################################################

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model) # loss function = MSE

In [ ]:
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')

In [ ]:
import math
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) # 10% of train data for warm-up

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          optimizer_class=torch.optim.Adam,
          optimizer_params={'lr': learn_rate},
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

In [ ]:
##############################################################################
# Load the stored model and evaluate its performance on STS benchmark dataset
##############################################################################
model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
test_evaluator(model, output_path=model_save_path)

0.7365893217230949

## Classification task

In [ ]:
!pip install wget

import wget
import os

print('***** Downloading dataset ...')

# The URL for the dataset zip file.
url = 'https://nlp.stanford.edu/projects/snli/snli_1.0.zip'

# Download the file (if we haven't already)
if not os.path.exists('./snli_1.0.zip'):
    wget.download(url, './snli_1.0.zip')

***** Downloading dataset ...


In [ ]:
!unzip snli_1.0.zip

Archive:  snli_1.0.zip
   creating: snli_1.0/
  inflating: snli_1.0/.DS_Store      
   creating: __MACOSX/
   creating: __MACOSX/snli_1.0/
  inflating: __MACOSX/snli_1.0/._.DS_Store  
 extracting: snli_1.0/Icon           
  inflating: __MACOSX/snli_1.0/._Icon  
  inflating: snli_1.0/README.txt     
  inflating: __MACOSX/snli_1.0/._README.txt  
  inflating: snli_1.0/snli_1.0_dev.jsonl  
  inflating: snli_1.0/snli_1.0_dev.txt  
  inflating: snli_1.0/snli_1.0_test.jsonl  
  inflating: snli_1.0/snli_1.0_test.txt  
  inflating: snli_1.0/snli_1.0_train.jsonl  
  inflating: snli_1.0/snli_1.0_train.txt  
  inflating: __MACOSX/._snli_1.0     


In [ ]:
# import pandas as pd
# df_train = pd.read_json(r'/content/snli_1.0/snli_1.0_train.jsonl', lines=True)
# df_train.to_csv(r'/content/snli_1.0_train.csv', index = None)

# df_test = pd.read_json(r'/content/snli_1.0/snli_1.0_test.jsonl', lines=True)
# df_test.to_csv(r'/content/snli_1.0_test.csv', index = None)

# df_dev = pd.read_json(r'/content/snli_1.0/snli_1.0_dev.jsonl', lines=True)
# df_dev.to_csv(r'/content/snli_1.0_dev.csv', index = None)

In [ ]:
label2int = {"contradiction": 0,
             "entailment": 1,
             "neutral": 2}

In [ ]:
train_path = '/content/snli_1.0_train.jsonl'
train_samples=[]

df_class = pd.read_json("snli_1.0/snli_1.0_train.jsonl", lines=True)
df_class

# index = df_class[df_class['gold_label'] == '-'].index
# df_class.drop(index, inplace=True)

,annotator_labels,captionID,gold_label,pairID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,[neutral],3416050480.jpg#4,neutral,3416050480.jpg#4r1n,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,A person is training his horse for a competition.,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
1,[contradiction],3416050480.jpg#4,contradiction,3416050480.jpg#4r1c,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is at a diner, ordering an omelette.",( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
2,[entailment],3416050480.jpg#4,entailment,3416050480.jpg#4r1e,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is outdoors, on a horse.","( ( A person ) ( ( ( ( is outdoors ) , ) ( on ...",(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
3,[neutral],2267923837.jpg#2,neutral,2267923837.jpg#2r1n,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,They are smiling at their parents,( They ( are ( smiling ( at ( their parents ) ...,(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VB...
4,[entailment],2267923837.jpg#2,entailment,2267923837.jpg#2r1e,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,There are children present,( There ( ( are children ) present ) ),(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NN...
...,...,...,...,...,...,...,...,...,...,...
550147,[contradiction],2267923837.jpg#3,contradiction,2267923837.jpg#3r1c,Four dirty and barefooted children.,( ( ( ( Four dirty ) and ) ( barefooted childr...,(ROOT (NP (NP (CD Four) (NNS dirty)) (CC and) ...,four kids won awards for 'cleanest feet',( ( four kids ) ( ( won awards ) ( ( ( for ` )...,(ROOT (S (NP (CD four) (NNS kids)) (VP (VBD wo...
550148,[neutral],2267923837.jpg#3,neutral,2267923837.jpg#3r1n,Four dirty and barefooted children.,( ( ( ( Four dirty ) and ) ( barefooted childr...,(ROOT (NP (NP (CD Four) (NNS dirty)) (CC and) ...,"four homeless children had their shoes stolen,...",( ( ( ( ( ( four ( homeless children ) ) ( had...,(ROOT (S (S (NP (CD four) (JJ homeless) (NNS c...
550149,[neutral],7979219683.jpg#2,neutral,7979219683.jpg#2r1n,A man is surfing in a bodysuit in beautiful bl...,( ( A man ) ( ( is ( surfing ( in ( ( a bodysu...,(ROOT (S (NP (DT A) (NN man)) (VP (VBZ is) (VP...,A man in a bodysuit is competing in a surfing ...,( ( ( A man ) ( in ( a bodysuit ) ) ) ( ( is (...,(ROOT (S (NP (NP (DT A) (NN man)) (PP (IN in) ...
550150,[contradiction],7979219683.jpg#2,contradiction,7979219683.jpg#2r1c,A man is surfing in a bodysuit in beautiful bl...,( ( A man ) ( ( is ( surfing ( in ( ( a bodysu...,(ROOT (S (NP (DT A) (NN man)) (VP (VBZ is) (VP...,A man in a business suit is heading to a board...,( ( ( A man ) ( in ( a ( business suit ) ) ) )...,(ROOT (S (NP (NP (DT A) (NN man)) (PP (IN in) ...


In [ ]:
print(df_class['gold_label'])

0               neutral
1         contradiction
2            entailment
3               neutral
4            entailment
              ...      
550147    contradiction
550148          neutral
550149          neutral
550150    contradiction
550151       entailment
Name: gold_label, Length: 550152, dtype: object


In [ ]:
df_class.gold_label.value_counts()

entailment       183416
contradiction    183187
neutral          182764
-                   785
Name: gold_label, dtype: int64

In [ ]:
print(len(df_class.gold_label.index))

550152


In [ ]:
# train_class = []
# delete = []

# for i in df_class.index:
#   print(i)
  
#   if df_class['gold_label'][i] == 'contradiction':
#     label_id = 0
#     inp_sample = InputExample(texts=[df_class['sentence1'][i], df_class['sentence2'][i]], label=label_id)
#     train_class.append(inp_sample)
#   elif df_class['gold_label'][i] == 'entailment':
#     label_id = 1
#     inp_sample = InputExample(texts=[df_class['sentence1'][i], df_class['sentence2'][i]], label=label_id)
#     train_class.append(inp_sample)
#   elif df_class['gold_label'][i] == 'neutral':
#     label_id = 2
#     inp_sample = InputExample(texts=[df_class['sentence1'][i], df_class['sentence2'][i]], label=label_id)
#     train_class.append(inp_sample)
#   else:
#     delete_id = 3
#     inp_sample =  InputExample(texts=[df_class['sentence1'][i], df_class['sentence2'][i]], label=delete_id)
#     delete.append(inp_sample)    

In [ ]:
## here we get rid of the "-" label

train_class = []

for i in df_class.index:
  #print(i)
  
  if df_class['gold_label'][i] == 'contradiction':
    label_id = 0
    
  elif df_class['gold_label'][i] == 'entailment':
    label_id = 1
    
  elif df_class['gold_label'][i] == 'neutral':
    label_id = 2
  
  inp_sample = InputExample(texts=[df_class['sentence1'][i], df_class['sentence2'][i]], label=label_id)
  train_class.append(inp_sample)
  #print(inp_sample)

In [ ]:
## previous try

# train_path = '/content/snli_1.0_dev.csv'
# train_samples=[]
# with open(train_path, newline='') as train:
#     columns=['annotator_labels',
#              'captionID',
#              'gold_label',
#              'pairID',
#              'sentence1', 'sentence1_binary_parse', 'sentence1_parse',
#              'sentence2', 'sentence2_binary_parse', 'sentence2_parse']
#     df = csv.DictReader(train, delimiter='|', fieldnames=columns, quoting=csv.QUOTE_NONE)
#     for row in df:
#         inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['gold_label'])
#         train_samples.append(inp_example)
#         #print(inp_example)

In [ ]:
test_path = 'snli_1.0/snli_1.0_test.jsonl'
df_class_test = pd.read_json(test_path, lines=True)

test_class=[]

for i in df_class_test.index:
  if df_class_test['gold_label'][i] == 'contradiction':
    label_id = 0
  elif df_class_test['gold_label'][i] == 'entailment':
    label_id = 1
  elif df_class_test['gold_label'][i] == 'neutral':
    label_id = 2
  inp_sample = InputExample(texts=[df_class_test['sentence1'][i], df_class_test['sentence2'][i]], label=label_id)
  test_class.append(inp_sample)
  #print(inp_sample)

In [ ]:
dev_path = 'snli_1.0/snli_1.0_dev.jsonl'

df_class_dev = pd.read_json(dev_path, lines=True)

dev_class = []

for i in df_class_dev.index:
  if df_class_dev['gold_label'][i] == 'contradiction':
    label_id = 0
  elif df_class_dev['gold_label'][i] == 'entailment':
    label_id  = 1
  elif df_class_dev['gold_label'][i] == 'neutral':
    label_id = 2
  inp_sample = InputExample(texts=[df_class_dev['sentence1'][i], df_class_dev['sentence2'][i]], label=label_id)
  dev_class.append(inp_sample)
  #print(inp_sample)

In [ ]:
# Read the dataset
train_batch_size = 16

model_save_path = './training_nli_'+model_name.replace("/", "-")+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [ ]:
## DOING IT WITH ALL THE DATASET
#train_dataloader = DataLoader(train_class, shuffle=True, batch_size=train_batch_size)

## DOING IT WITH A SUBSET
train_dataloader = DataLoader(train_class[0:200000], shuffle=True, batch_size=train_batch_size)

#     :param model: SentenceTransformer model => BERT base
#     :param sentence_embedding_dimension: Dimension of your sentence embeddings => shape
#     :param num_labels: Number of different labels => 3
#     :param concatenation_sent_rep: Concatenate vectors u,v for the softmax classifier? => T
#     :param concatenation_sent_difference: Add abs(u-v) for the softmax classifier? => T
#     :param concatenation_sent_multiplication: Add u*v for the softmax classifier? => F
#     :param loss_fct: Optional: Custom pytorch loss function. If not set, uses nn.CrossEntropyLoss() => T

################################################################################
# creating custom model, by using mean pooling of the word embeddings given as input
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
################################################################################

train_loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=3)

In [ ]:
dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_class, batch_size=train_batch_size, name='snli-dev')

In [ ]:
# Configure the training
num_epochs = 1

warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up

In [ ]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=dev_evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path
          )

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/34385 [00:00<?, ?it/s]

## Evaluation with snli-test

In [ ]:
model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_class, name='snli-test')
test_evaluator(model, output_path=model_save_path)

## Evaluation with STS-benchmark test and dev for eval

In [ ]:
import scipy

#dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, batch_size=train_batch_size, main_similarity='Cosine', name='sts-dev')
dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, batch_size=train_batch_size, name='sts-dev')

## to be implemented!!!
dev_evaluator = scipy.stats.spearmanr(a, b=None, axis=0, nan_policy='propagate', alternative='two-sided')

In [ ]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=dev_evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path
          )


In [ ]:
model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
test_evaluator(model, output_path=model_save_path)

In [ ]:
## MOUNT DRIVE TO SAVE MODEL

# from google.colab import drive
# drive.mount('/content/drive')

## Semantic search

In [ ]:
## LINK
# https://www.kaggle.com/rmisra/news-category-dataset/download

In [ ]:
## DOESN'T WORK WITH LINK

# print('Downloading dataset...')

# # The URL for the dataset zip file.
# url = 'https://www.kaggle.com/rmisra/news-category-dataset?select=News_Category_Dataset_v2.json'

# # Download the file (if we haven't already)
# if not os.path.exists('./news.zip'):
#     wget.download(url,'./news.zip')

print('Uploading dataset to directory...')

In [ ]:
!unzip news.zip

In [ ]:
# We use the best model to encode all passages, so that we can use it with sematic search
#model_name = '/content/best_models/2.3-task'
model_name = '/content/drive/MyDrive/Colab Notebooks/training_nli_'
encoder = SentenceTransformer(model_name)
top_k = 5  # number of passages we want to retrieve with the bi-encoder

In [ ]:
# import gzip

# news = []
# with gzip.open(url, 'rt', encoding='utf8') as fIn:
#     for line in fIn:
#         data = json.loads(line.strip())
#         for paragraph in data['short_description']:
#             # We encode the passages as [title, text]
#             news.append([data['headlines'], paragraph])

In [ ]:
news = []

with open('/content/News_Category_Dataset_v2.json') as fIn:
    for line in fIn:
        data = json.loads(line.strip())
        #print(data)
        for paragraph in data['short_description']:
            # We encode the passages as [title, text]
            news.append([data['headline'], paragraph])

In [ ]:
#corpus_embeddings = encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)
corpus_embeddings = encoder.encode(news, convert_to_tensor=True, show_progress_bar=True)

In [ ]:
## COMMENT THIS

# while True:
#     input = input("Please enter a question: ")

#     # Encode the query using the encoder and find potentially relevant passages
#     start_time = time.time()
#     question_embedding = encoder.encode(input, convert_to_tensor=True)
#     hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
#     hits = hits[0]  # Get the hits for the first query

#     end_time = time.time()

#     # Output of top-k hits
#     print("Input question:", query)
#     print("Results (after {:.3f} seconds):".format(end_time - start_time))
#     for hit in hits:
#         print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']]))

#     print("\n\n========\n")

In [ ]:
top_k = min(5, len(corpus))

input = input("Please enter a question: ")

query_embedding = encoder.encode(input, convert_to_tensor=True)

# We use cosine-similarity and torch.topk to find the highest 5 scores
cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
top_results = torch.topk(cos_scores, k=top_k)

print("\n\n======================\n\n")
print("Input:", input)
print("\nTop 5 most similar sentences in corpus:")

for score, idx in zip(top_results[0], top_results[1]):
      print(corpus[idx], "(Score: {:.4f})".format(score))